#**Data Cleaning**


Pandas Pipeline
1.   Impor the data.
2.   Set the data types. (as function)
3.   Remove the outlier. (as function)





```
df.pipe(set_type).pipe(remove_out, min_row_country=10)
```



In [2]:
# import libraries
import pandas as pd
import numpy as np

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

csv_names = ['products', 'orders', 'orderlines', 'brands']

# Google Drive file ids


files_id = {
    'products':"1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU",
    'orders':"1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ",
    'orderlines':"1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG",
    'brands':"1m1ThDDIYRTTii-rqM5SEQjJ8McidJskD" 
} 

# Read data from Google Drive
products = pd.read_csv(gd_path(files_id['products']), sep=",")
orders = pd.read_csv(gd_path(files_id['orders']), sep=",")
orderlines = pd.read_csv(gd_path(files_id['orderlines']), sep=",")
brands = pd.read_csv(gd_path(files_id['brands']), sep=",")

# 1 Products [non ok]

**products.csv**
* `sku` – stock keeping unit: a unique identifier for each product
* `name` – product name
* `desc` – product description
* `in_stock` – whether or not the product was in stock at the moment of the data extraction
* `type` – a numerical code for product type

In [35]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


###Looking for Duplicates


### Functions

In [36]:
def start_pipeline(dataf):
    return dataf.copy()

def remove_dup(dataf):
    return dataf.drop_duplicates()

def check(df, operator, column_name, length_of_element,):
    if operator == len:  # Call with attribute len for lenght 
      i = '.'   
    else: 
      i = '\.'          # Call with attribute '' for dot count 
    return df[column_name].str.count(i).eq(length_of_element).sum()   


# Remove "porducts_1sp" !!!
def case_price(column, lenght, dots_count):
  return products_1sp[(products_1sp[column].str.count('.').eq(lenght)) & (products_1sp[column].str.count('\.').eq(dots_count))]

### Preprocessing 
Check the size and drop duplicates.

In [37]:
products.nunique(), products.shape

(sku            10579
 name           10373
 desc            7098
 price           2690
 promo_price     4614
 in_stock           2
 type             126
 dtype: int64,
 (19326, 7))

In [38]:
products_1sp = products.pipe(start_pipeline).pipe(remove_dup)

#### **Price analyses**

In [39]:
pd.set_option('display.max_colwidth', None) # Display full cell wight 

In [40]:
products_1sp[["name","price","promo_price"]].sample(10)

,name,price,promo_price
14838,(Open) Silver Rain Design mStand Support,59.99,377.671
15122,"Open - Moshi iGlaze Case for iPad Pro 129 ""Transparent",40,260.701
15920,Synology DS1517 + | 16GB RAM | 40TB (5x8TB) WD Red,27.798.392,21.867.459
2399,Devolo Wifi Repeater 24 GHz,399.905,329.846
2073,Pack QNAP TS-251 | 8TB Seagate Desktop,5.108.693,4.909.902
18726,Open - Tado V2 Intelligent Heating Thermostat,1.799.899,1.512.253
15383,LCD Screen Repair iPad Mini 3,1.699.905,1.699.905
12637,"Speck SeeThru Case Macbook Pro 13 ""Blue Calypso",49.9,399.905
11861,"Blue WD 3TB Hard Drive 35 ""Mac and PC",129,895.945
15344,iPhone 5c front camera repair,599.906,599.906


In [41]:
products_1sp[products_1sp.price == products_1sp.promo_price].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 60 to 19306
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          204 non-null    object
 1   name         204 non-null    object
 2   desc         203 non-null    object
 3   price        204 non-null    object
 4   promo_price  204 non-null    object
 5   in_stock     204 non-null    int64 
 6   type         200 non-null    object
dtypes: int64(1), object(6)
memory usage: 12.8+ KB


In [42]:
products_1sp[products_1sp.price.str.count('.') != products_1sp.promo_price.str.count('.')].sample(5)

,sku,name,desc,price,promo_price,in_stock,type
16914,APP2167,"Apple iPad Leather Smart Cover Case Pro ""Candy Brown 10.5",Intelligent quality leather case with function on / off 105-inch iPad Pro,85,789.997,0,12635403
1993,PAC0736,Samsung SSD 850 expansion kit EVO 1TB for Mac mini 2011/2012,SSD upgrade kit for Mac mini 1TB 2011 Late 2012 Mid tools,444.98,3.225.848,1,1433
11741,PAC1381,Pack QNAP TS-228 | WD 6TB Network,NAS + 6TB (2x3TB) WD Network for Mac and PC.,398.99,3.451.791,0,12175397
15390,REP0300,Full screen repair iPhone 6 Plus,Repair service including parts and labor for iPhone 6 Plus,99.99,999.899,0,"1,44E+11"
14830,OWC0039-A,Open - Mac OWC Memory 4GB DDR3 1600MHz SO-DIMM,4GB RAM MacBook Pro and Mac mini iMac (2012).,40.99,326.676,0,1298


In [43]:
# Investigate the cases
products_1sp[products_1sp.price.notna() & products_1sp.price.str.match(r'^\d{2}\.\d{3}$')].sort_values(by="sku").head(5)

,sku,name,desc,price,promo_price,in_stock,type
13017,CRU0044-A,Open - Crucial MX300 275GB SSD Disk,SSD 275GB SATA Hard Disk 3.0 (6Gb / s) for Mac and PC,91.989,918.726,0,1298
14836,DLK0068-A,Open - D-Link GO-SW-5E 5 port Fast Ethernet Switch,D-Link GO-SW-5E 5 port 10/100 Mbps Fast Ethernet,89.903,63.555,0,1298
1196,DLK0077,D-Link DGS-1005D Gigabit Switch 5-port Unmanaged,D-Link Gigabit Switch 5-port high performance.,25.999,149.895,0,1334
11446,DLK0123,D-Link GO-RT-N300 Wireless Router N300 Easy,Easy access router and speeds of 300 Mbps.,24.785,299.899,0,1334
11404,DLK0128,D-Link DES-1005P PoE Ethernet Switch,compact switch with 5 Ethernet (PoE 1).,48.752,589.899,0,1334


#### **Clean by cases**

The columsn `price` and `prome_price` seem to have multiple mistake. 

Range of "." count ist 0-2.
Range of digits in a numer 1-11.

```
CASE 1
# - 352.559.394 (10)  / 100000
-  12.808.289 (138) / 100000
-   6.999.003 (229) / 100000
	
r'^\d{1,3}\.\d{3}\.\d{3}$'

CASE 2	
-     164.923 (139) / 10000 

r'^\d{3}\.\d{3}$'

CASE 3
-      91.989 (23)  > float
-       9.082 (3)   > float

-     679.99 > float
-      79.99 > float
-       9.99 > float
 
-    2117.5 (35) > float
-     119.9 (113)> float 
-      19.9 (347)> float
-       9.9 (11) > float 

r'^\d{1,4}\.\d{1,3}$'

CASE 4
- 11019 (42)   > float
-  2859 (1132) > float
-   209 (1112) > float
-    35        > float
-     9        > float 

r'^\d{1,5}$'

```

In [44]:
#for i in range 11

check(products_1sp, len, 'price', 8)

1

####**Sku**

In [45]:
len(products_1sp.sku.unique())

10579

In [46]:
products_1sp[products_1sp.sku.str.slice(0,3) == "REP"].sort_values(by="desc").sample(5)

,sku,name,desc,price,promo_price,in_stock,type
15329,REP0273,volume button repair on vibration control and flexo iPhone 5s,Repair service including parts and labor for iPhone 5s,6.999.003,69.99,0,"1,44E+11"
15399,REP0333,Full Screen Repair iPad 2,Repair service including parts and labor for iPad 2,159.99,159.99,0,"1,44E+11"
15439,REP0382,Home button repair iPad Mini 3,Repair service including parts and labor for iPad Mini 3,6.999.003,69.99,0,"1,44E+11"
375,REP0051,Load Connector Repair iPhone 4,Repair service including parts and labor for iPhone 4,39.99,399.893,0,"1,44E+11"
15364,REP0270,Load Connector Repair iPhone 5,Repair service including parts and labor for iPhone 5,49.99,499.899,0,"1,44E+11"


In [47]:
products_1sp["brands"] = products_1sp.sku.str.slice(0,3)
products_2sp = products_1sp.merge(brands, how="inner", left_on="brands", right_on="short")[["sku","long", "name", "desc", "price", "promo_price", "type"]].copy()
products_2sp

,sku,long,name,desc,price,promo_price,type
0,RAI0007,Rain Design,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,8696
1,RAI0014,Rain Design,Rain Design iSlider iPad Support,Anodized aluminum support multiposition iPad and iPad mini.,49.99,349.932,1216
2,RAI0015,Rain Design,Rain Design MTower support MacBook Air / Pro,Aluminum support for MacBook Pro and MacBook Air,59.99,499.899,8696
3,RAI0016,Rain Design,Rain Design iLevel2 MacBook Support,Adjustable aluminum bracket height compatible with all MacBook,84.99,669.904,8696
4,RAI0018,Rain Design,"Rain Design Mbase Support for iMac 27 """,Minimalist support lifting drawer iMac 27,84.99,749.898,8696
...,...,...,...,...,...,...,...
10364,RYV0009,Ryval,Ryval Base Magnetic Qi Wireless Charging Office,Wireless charging base Ryval magnetic and Qi technology ideal for iPhone X / 8Plus / 8 at home and in the office,49.99,449.878,5399
10365,RYV0010,Ryval,Ryval magnetic Qi Wireless Car Charger,Ryval magnetic car charger and Qi Wireless technology is ideal for iPhone X / 8Plus / 8,49.99,449.878,5399
10366,MTF0008,Mistify,Mistify Clean Screens Natural 500ml.,Spray cleaning screens and keyboards.,14.99,13.99,12085400
10367,SDE0007,SDesign,SDesign iPhone charging dock QI X / 8 / Apple Watch (75-10W),Base load compatible with iPhone and Apple Watch supports multiload,69.99,65.99,5399


### Status
`price`, `promo_price` 
are not OK

`type, name, desc`
keep, no cleaning required

# 2 Brands [ok]

**brands.csv**
* `short` – the 3-character code by which the brand can be identified in the first 3 characters of products.sku
* `long` – brand name

In [33]:
brands.sample(10)

,short,long
73,KOO,Koogeek
154,STC,Stacked
66,JOB,Joby
58,IFX,iFixit
168,TRA,Trascend
91,MIN,Minibatt
37,ENV,Unknown
16,BOO,Booq
84,LUN,Lunatik
156,STK,Stikgo


In [34]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


### Status

All datatype are OK

# 3 Orders [ok]

**orders.csv** – Every row in this file represents an order.
* `order_id` – a unique identifier for each order
* `created_date` – a timestamp for when the order was created
* `total_paid` – the total amount paid by the customer for this order, in euros
state 
 * “Shopping basket” – products have been placed in the shopping basket, but the order has not been processed yet.
 * “Pending” – the shopping basket has been processed, but payment confirmation is pending.
 * “Completed” – the order has been placed and paid, and the transaction is completed.
 * “Cancelled” – the order has been cancelled and the payment returned to the 
customer.

In [49]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [149]:
orders.sample(5)

,order_id,created_date,total_paid,state
209148,509639,2018-02-11 15:32:23,12.98,Cancelled
3596,303064,2017-01-05 11:14:02,9.99,Shopping Basket
168031,468376,2017-12-20 21:07:23,1159.00,Shopping Basket
72502,372048,2017-07-03 16:59:03,0.00,Place Order
103100,402685,2017-09-21 12:52:59,0.00,Place Order


In [55]:
orders.nunique()

order_id        226909
created_date    224828
total_paid       31236
state                5
dtype: int64

In [56]:
len(orders.order_id.unique())

226909

All `order_is` are unique.

In [67]:
orders_1sp = orders.copy()
orders_1sp["created_date"] = pd.to_datetime(orders.created_date)

In [66]:
orders_1sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226909 non-null  int64         
 1   created_date  226909 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 6.9+ MB


In [144]:
orders_1sp.sample(5)

,order_id,created_date,total_paid,state
136370,436065,2017-11-23 22:31:24,1698.39,Completed
215807,516298,2018-02-23 07:32:19,69.99,Shopping Basket
62961,362504,2017-06-12 12:27:06,91.98,Completed
159592,459817,2017-12-11 13:23:55,59.99,Shopping Basket
63742,363286,2017-06-07 18:48:15,0.00,Place Order


In [143]:
orders_1sp[orders_1sp.state == "Completed"].total_paid.describe()

count    46605.000000
mean       341.381381
std        625.528004
min          0.000000
25%         50.980000
50%        113.230000
75%        333.970000
max      13387.780000
Name: total_paid, dtype: float64

### Status
All datatype are OK
Pricerange is not OK

#4 Orderlines

**orderlines.csv** – Every row represents each one of the different products involved in an order.
* `id` – a unique identifier for each row in this file
* `id_order` – corresponds to orders.order_id
* `product_id` – an old identifier for each product, nowadays not in use
* `product_quantity` – how many units of that product were purchased on that order
* `sku` – stock keeping unit: a unique identifier for each product
* `unit_price` – the unitary price (in euros) of each product at the moment of placing that order
date 
* `timestamp` for the processing of that product

In [68]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [109]:
orderlines[orderlines['id_order'] == 461458], orders_1sp[orders_1sp['order_id'] == 461458]

(             id  id_order  product_id  product_quantity      sku unit_price  \
 199773  1492821    461458           0                 1  MIN0011      13.99   
 
                        date  
 199773  2017-12-13 07:37:44  ,
         order_id        created_date  total_paid            state
 161200    461458 2017-12-13 07:37:44       13.99  Shopping Basket)

* `orders`:`created_date` is the same as `orderlines`:`date`
 
* `orders`:`order_id` is the same as `orderlines`:`id_order`






In [83]:
orderlines_1sp = orderlines.copy()
orderlines_1sp["date"] = pd.to_datetime(orderlines.date)
orderlines_1sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


In [121]:
orderlines[orderlines.product_quantity ==2]

,id,id_order,product_id,product_quantity,sku,unit_price,date
20,1119136,299558,0,2,WDT0141,112.99,2017-01-01 02:24:33
58,1119213,299595,0,2,OWC0153-4,286.99,2017-01-01 09:45:56
77,1119251,296284,0,2,OWC0007,39.99,2017-01-01 10:41:53
86,1119262,299600,0,2,BEL0229,18.99,2017-01-01 10:55:02
93,1119275,299616,0,2,WDT0141,107.34,2017-01-01 11:05:56
...,...,...,...,...,...,...,...
293921,1650108,527352,0,2,APP0927,13.99,2018-03-14 13:29:58
293925,1650113,527358,0,2,APP0927,13.99,2018-03-14 13:31:45
293947,1650152,527370,0,2,APP0698,9.99,2018-03-14 13:40:36
293953,1650161,527375,0,2,OWC0255,46.99,2018-03-14 13:42:37


In [72]:
orderlines.id_order.nunique()

204855

Different ammount of unique `id_order`

Orders: 226909
Orderlines: 204855

In [77]:
orderlines.query('product_quantity == 1').id_order.nunique()

194780

In [78]:
orderlines.query('product_quantity != 1').id_order.nunique()

14734

In [80]:
orderlines.sku.nunique()

7951

# Combine Tables

In [153]:
all_commbine = (
orders_1sp
 .merge(
    orderlines_1sp, 
    how = "left", 
    left_on ="order_id", 
    right_on ="id_order")
 .merge(
    products_2sp, 
    how = "left",
    left_on = "sku",
    right_on = "sku")
 .copy()
)

all_commbine.sample(10)

,order_id,created_date,total_paid,state,id,id_order,product_id,product_quantity,sku,unit_price,date,long,name,desc,price,promo_price,type
86186,368552,2017-06-24 07:39:11,749.00,Shopping Basket,1274088.0,368552.0,0.0,1.0,AP20180,749.00,2017-06-24 07:39:11,Apple,Like new - Apple iPhone 7 128GB Black,Free Apple iPhone 7 refitted 128GB Color Black Bright,749,6.489.956,85641716
66233,352674,2017-05-04 14:16:32,35.99,Shopping Basket,1241551.0,352674.0,0.0,1.0,APP1683,35.99,2017-05-04 14:16:32,Apple,Apple iPhone Silicone Case Cover 7 Rosa Arena,Ultrathin silicone case and microfiber premium for iPhone 7,39,330.003,11865403
26048,320479,2017-02-06 13:46:29,39.99,Shopping Basket,1172456.0,320479.0,0.0,1.0,STM0085,39.99,2017-02-06 13:46:29,STM,Dux STM rigid Case Black iPad Air 2,Resistant sleeve with original design and magnetic closure for iPad Air 2.,59.96,369.897,12635403
236659,478273,2017-12-31 20:55:36,0.00,Shopping Basket,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
83428,366378,2017-06-16 18:28:36,199.99,Shopping Basket,1269595.0,366378.0,0.0,1.0,QNA0204,199.99,2017-06-16 18:28:36,QNAP,QNAP TS-231P NAS,2-bay NAS server with 2 LAN ports and a powerful processor for home and small business Mac and PC,216.59,2.069.899,12175397
194893,451373,2017-11-29 05:43:36,3772.56,Shopping Basket,1466609.0,451373.0,0.0,1.0,MMW0018,22.99,2017-11-29 05:43:36,My MW,"My MW Case MacBook Pro 15 ""(Late 2016) Black",Avoid shock and damage to your MacBook Pro 15-inch end of 2016 thanks to the foam LRPu,29.99,20.99,13835403
311907,524684,2018-03-10 08:47:01,3478.99,Shopping Basket,1644069.0,524684.0,0.0,1.0,SAT0054,74.99,2018-03-10 08:47:01,Satechi,Satechi Hub Multiport USB-C 4K HDMI Space Gray,Aluminum adapter with USB Type-C HDMI 4K three ports USB Type-A Ethernet input and SD card reader micro SD for Mac and PC.,89.99,69.99,12585395
271260,499634,2018-01-27 14:18:03,0.00,Shopping Basket,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
234707,476687,2017-12-29 19:15:26,0.00,Place Order,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
310786,524343,2018-03-10 00:00:19,328.98,Place Order,1642921.0,524343.0,0.0,1.0,LAC0200,324.99,2018-03-10 00:00:19,LaCie,LaCie Rugged Hard Disk Raid 4TB Thunderbolt USB 3.0,"rugged, compact 4TB hard drive with USB 3.0 and Thunderbolt RAID system for Mac and PC",399,3.249.939,11935397


In [151]:
(
all_commbine[(all_commbine.product_quantity == 1)
           & (all_commbine.state == "Completed")
           & (all_commbine.total_paid > 10) 
           & (all_commbine.total_paid < 1000) ]
            [["total_paid",
              "unit_price", 
              "price", 
              "promo_price", 
              "product_quantity"]]
              .sort_values(by="total_paid")
              .drop_duplicates()
)

,total_paid,unit_price,price,promo_price,product_quantity
21515,10.44,10.44,14.99,109.868,1.0
60114,10.48,5.49,6.99,54.898,1.0
118778,10.63,6.64,9.95,69.902,1.0
22619,10.68,5.69,5.99,39.906,1.0
22238,10.68,5.69,5.99,59.895,1.0
...,...,...,...,...,...
278149,999.00,999.00,1059,1.027.002,1.0
289109,999.36,992.37,1249.95,9.093.682,1.0
167039,999.76,154.00,179,1.610.001,1.0
167038,999.76,845.76,899,872.001,1.0
